In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
np.set_printoptions(precision=2)
import matplotlib.pyplot as plt
import copy as cp
import sys, json, pickle
PROJECT_PATHS = ['/home/nbuckman/Dropbox (MIT)/DRL/2020_01_cooperative_mpc/mpc-multiple-vehicles/', '/Users/noambuckman/mpc-multiple-vehicles/']
for p in PROJECT_PATHS:
    sys.path.append(p)
import src.traffic_world as tw
import src.multiagent_mpc as mpc
import src.car_plotting_multiple as cmplot
import src.solver_helper as helper
import src.vehicle as vehicle

In [8]:
i_mpc_start = 1
i_mpc = i_mpc_start
log_directory = '/home/nbuckman/mpc_results/f509-425f-20200907-153800/'

In [9]:
with open(log_directory + "params.json",'rb') as fp:
    params = json.load(fp)

In [13]:
n_rounds_mpc = params['n_rounds_mpc']
number_ctrl_pts_executed = params['number_ctrl_pts_executed']


xamb_actual, uamb_actual = np.zeros((6, n_rounds_mpc*number_ctrl_pts_executed + 1)), np.zeros((2, n_rounds_mpc*number_ctrl_pts_executed)) 
xothers_actual = [np.zeros((6, n_rounds_mpc*number_ctrl_pts_executed + 1)) for i in range(params['n_other'])]
uothers_actual = [np.zeros((2, n_rounds_mpc*number_ctrl_pts_executed)) for i in range(params['n_other'])]    

In [33]:
actual_t = 0
last_mpc_i = 104
for i_mpc_start in range(1,last_mpc_i+2):
    previous_mpc_file = folder + 'data/mpc_%02d'%(i_mpc_start - 1)
    xamb_executed, uamb_executed, _, all_other_x_executed, all_other_u_executed, _,  = mpc.load_state(previous_mpc_file, params['n_other'])
    all_other_u_mpc = all_other_u_executed
    uamb_mpc = uamb_executed
    previous_all_file = folder + 'data/all_%02d'%(i_mpc_start -1)
#     xamb_actual_prev, uamb_actual_prev, _, xothers_actual_prev, uothers_actual_prev, _ = mpc.load_state(previous_all_file, params['n_other'], ignore_des = True)
    t_end = actual_t+number_ctrl_pts_executed+1
    xamb_actual[:, actual_t:t_end] = xamb_executed[:,:number_ctrl_pts_executed+1]
    uamb_actual[:, actual_t:t_end] = uamb_executed[:,:number_ctrl_pts_executed+1]
    for i in range(len(xothers_actual_prev)):
        xothers_actual[i][:, actual_t:t_end] = all_other_x_executed[i][:,:number_ctrl_pts_executed+1]
        uothers_actual[i][:, actual_t:t_end] = all_other_u_executed[i][:,:number_ctrl_pts_executed+1]
#     print(xamb_actual[0,:t_end])

# print(" ")
    file_name = folder + "data/"+'all_%02d'%(i_mpc_start-1)        
    mpc.save_state(file_name, xamb_actual, uamb_actual, None, xothers_actual, uothers_actual, None, end_t = actual_t+number_ctrl_pts_executed+1)
    actual_t += number_ctrl_pts_executed
print("Loaded initial positions from %s"%(previous_mpc_file))
print(xothers_actual[0][0,:t_end])

Loaded initial positions from /home/nbuckman/mpc_results/f509-425f-20200907-153800/data/mpc_104
[ 20.    22.22  24.46  26.69  28.93  31.16  33.4   35.63  37.87  40.1
  42.34  44.57  46.81  49.04  51.28  53.51  55.75  57.98  60.22  62.45
  64.69  66.92  69.16  71.39  73.63  75.86  78.1   80.33  82.57  84.8
  87.04  89.27  91.5   93.74  95.97  98.21 100.44 102.68 104.91 107.15
 109.38 111.62 113.85 116.09 118.32 120.56 122.79 125.03 127.26 129.5
 131.73 133.97 136.2  138.44 140.67 142.91 145.14 147.38 149.61 151.85
 154.08 156.32 158.55 160.79 163.02 165.26 167.49 169.73 171.96 174.2
 176.43 178.67 180.9  183.13 185.37 187.6  189.84 192.07 194.31 196.54
 198.78 201.01 203.25 205.48 207.72 209.95 212.19 214.42 216.66 218.89
 221.13 223.36 225.6  227.83 230.07 232.3  234.54 236.77 239.01 241.24
 243.48 245.71 247.95 250.18 252.42 254.65 256.89 259.12 261.36 263.59
 265.83 268.06 270.3  272.53 274.77 277.   279.24 281.47 283.71 285.94
 288.18 290.41 292.65 294.88 297.12 299.35 301.59 303.82